# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [3]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [4]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the Formula 1 World Championship in 2010. He was driving for the Red Bull Racing team.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [5]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [6]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

The 2006 F1 championship was won by Fernando Alonso, driving for Renault.


In [7]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

Lewis Hamilton won the 2019 F1 championship.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [8]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [9]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Drive: Lewis Hamilton
Team: Mercedes
Points: 413


In [10]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [11]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Neutral


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [12]:
from openai import OpenAI
import os

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0.7):
    messages = [
        {"role": "system", "content": "You are an expert on Khalil Gibran."},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

# Few-Shot Prompt Examples
few_shot_examples = [
    ("Who was Khalil Gibran?", "Khalil Gibran (1883-1931) was a Lebanese-American poet, writer, and artist, best known for his book 'The Prophet'. His works explore themes of love, spirituality, and human connection, blending Eastern and Western philosophies."),
    ("Write a poetic passage in the style of Gibran about wisdom.", "Wisdom is the lantern of the soul, lighting the path of the seeker. It is not spoken in words, but in the silence between them, where truth whispers to the heart of the knowing.")
]

# Constructing Few-Shot Prompt
few_shot_prompt = "".join([f"User: {ex[0]}\nAssistant: {ex[1]}\n" for ex in few_shot_examples])

# New Query for the Model
new_query = "What is the central theme of 'The Prophet'?"
final_prompt = few_shot_prompt + f"User: {new_query}\nAssistant: "

# Get Response
response = get_completion(final_prompt)
print(response)


The central theme of 'The Prophet' by Khalil Gibran is the exploration of various aspects of life through the voice of a prophet named Almustafa. The book delves into themes such as love, marriage, work, freedom, and spirituality, offering timeless wisdom and guidance on how to navigate the complexities of human existence. Gibran's poetic and philosophical approach in 'The Prophet' has made it a beloved classic that continues to resonate with readers around the world.


In experimenting with Few-Shot Learning for responses about Khalil Gibran, the results varied based on prompt structure and context.

Findings:

Effective Variations: Providing clear, structured examples helped guide the model’s responses, ensuring accuracy and coherence.
Issues Observed: In some cases, GPT produced poetic but factually incorrect statements, especially when prompts were vague.
Hallucinations: Occasionally, GPT generated quotes falsely attributed to Gibran or misrepresented his philosophy.

Through this exercise, I learned that Few-Shot Learning is an effective technique for guiding GPT’s responses